In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)


train_data = train_datagen.flow_from_directory('data/train/',
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=256,
                                               subset='training',
                                               target_size=(100, 100))

valid_data = train_datagen.flow_from_directory('data/train/',
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=256,
                                               subset='validation',
                                               target_size=(100, 100))

test_data = train_datagen.flow_from_directory('data/test',
                                              color_mode='rgb',
                                              class_mode='categorical',
                                              batch_size=256,
                                              target_size=(100, 100))

Found 3161 images belonging to 10 classes.
Found 349 images belonging to 10 classes.
Found 1170 images belonging to 10 classes.


In [3]:
train_data.class_indices

{'Apple 5': 0,
 'Apricot 1': 1,
 'Avocado 1': 2,
 'Banana 3': 3,
 'Beans 1': 4,
 'Blackberrie 1': 5,
 'Cabbage white 1': 6,
 'Cactus fruit 1': 7,
 'Carrot 1': 8,
 'Corn 1': 9}

In [4]:
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential

In [5]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = (100,100,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.summary()

/opt/homebrew/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 98, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 47, 47, 128)    │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 128)    │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,888 (148.00 KB)

 Trainable params: 37,888 (148.00 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(10, activation = 'sigmoid'))

model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer='adam')
model.fit(train_data, validation_data= valid_data, epochs=5)

/opt/homebrew/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.3372 - loss: 2.0367 - val_accuracy: 0.6504 - val_loss: 1.0499
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.6893 - loss: 0.8477 - val_accuracy: 0.8281 - val_loss: 0.4135
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.8693 - loss: 0.3633 - val_accuracy: 0.9914 - val_loss: 0.1208
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.9257 - loss: 0.1598 - val_accuracy: 0.9885 - val_loss: 0.0332
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.9231 - loss: 0.0904 - val_accuracy: 0.9885 - val_loss: 0.0198


In [7]:
loss, accuracy = model.evaluate(test_data)
print(accuracy * 100)

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 363ms/step - accuracy: 0.9538 - loss: 0.0385
95.38461565971375


In [8]:
model.save('models/simple-cnn-84.h5')